In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
# from dotenv import load_dotenv

# API 키 정보 로드
# load_dotenv()

In [2]:
# from typing import Annotated, TypedDict
# from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, ToolMessage
# from langchain.callbacks.tracers import ConsoleCallbackHandler
from function_call_tool import tools

In [3]:
QUESTION_LIST = [
    "주문 조회해줘",
    "내 등급 조회",
    "스트라이프 셔츠 주문한거 언제와",
    "내 등급이랑 주문정보 알려줘", # multi tool use
    "한국의 수도가 어디야"
]

llm_invoke_callbacks = []
# llm_invoke_callbacks = [ConsoleCallbackHandler()]

tool_dict = {t.name:t for t in tools}

In [4]:
# from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

# LLM 초기화
# llm = ChatOpenAI(model="gpt-4o-mini")
# llm = ChatOllama(model="mistral:latest", temperature=0, base_url = "http://192.168.1.203:11434")
llm = ChatOllama(model="qwq:latest", temperature=0, base_url = "http://192.168.1.203:11434")

In [5]:
# 도구 이름 매핑
tool_dict = {t.name: t for t in tools}
llm_with_tools = llm.bind_tools(tools)

def invoke_model(query, llm):
    messages = [HumanMessage(content=query)]
    ai_msg = llm.invoke(messages)
    messages.append(ai_msg)
    
    # tool call 처리
    if ai_msg.tool_calls:
        for tool_call in ai_msg.tool_calls:
            selected_tool = tool_dict[tool_call['name']]
            print(f"* {selected_tool.name}() tool 사용중...")
            
            # tool 실행
            tool_result = selected_tool.invoke(tool_call['args'])
            
            # ToolMessage 생성 (tool_call_id 포함)
            tool_msg = ToolMessage(
                content=str(tool_result),
                tool_call_id=tool_call['id']
            )
            messages.append(tool_msg)
        
        # tool 결과를 포함하여 최종 응답 생성
        response2 = llm.invoke(messages)
        answer = response2.content
    else:
        answer = ai_msg.content
    
    return answer

In [6]:
for q in QUESTION_LIST:
    print(f"Q: {q}")
    # answer = invoke_model(q, llm)
    answer = invoke_model(q, llm_with_tools)
    print(f"A : {answer}\n")
    print("-"*50) 

Q: 주문 조회해줘
* get_order_info() tool 사용중...
A : <think>
좋아, 사용자가 주문 조회를 요청했을 때 get_order_info 함수를 호출해서 결과를 받았어. 이제 이 결과를 사용자에게 어떻게 전달해야 할까? 주문 내역이 두 개의 상품으로 되어 있으니, 각 상품의 이름과 가격을 명확하게 보여주면 좋겠어. 

먼저, 사용자에게 친절한 인사로 시작하는 게 좋을 것 같아. "주문하신 상품 정보입니다:" 같은 문구로 시작하고, 각 상품을 나열하는 형식으로 정리하면 이해하기 쉬울 거야. 가격은 원화이므로 정확히 표시해야 하고요.

또한, 배송 정보나 등급 관련 정보가 필요할 수도 있으니, 추가로 물어보는 것도 좋을 것 같아. 예를 들어 "배송 정보를 확인하시려면 상품명을 알려주세요"나 "등급 혜택을 확인하시려면 등급 조회도 가능합니다" 같은 문장을 추가하면 사용자의 추가 요청을 유도할 수 있겠지.

마지막으로 전체적인 흐름이 자연스럽고 친절해야 하니까, 문장의 끝에 감사의 말이나 도움이 더 필요하면 언제든지 문구를 넣는 것도 좋을 것 같아. 이렇게 하면 사용자의 질문에 충분히 답변하면서도 추가적인 도움을 제공할 수 있을 거야.
</think>

주문하신 상품 정보입니다:
1. 스트라이프 셔츠 - 12,000원
2. 트레이닝 바지 - 35,000원

배송 정보를 확인하시려면 상품명을 알려주시고, 등급 혜택을 확인하시려면 '등급 조회'를 말씀해주세요! 😊

--------------------------------------------------
Q: 내 등급 조회
* get_grade_info() tool 사용중...
A : <think>
Okay, the user asked to check their grade. I used the get_grade_info function, which doesn't require any parameters. The response from the tool was "VIP". So I need to